# UAV product workflow

In order for UAV data to be published in the DEPWS spatial library UAV data procedures require that the data has been processed using the approved methodology, and that the final product has been composited (where applicable) clipped, renamed with meta data updated.

This notebook facilities the processing of UAV images by:
 - Compositing grayscale images form the multispectral camera
 - Clipping images with an extent vector
 - Renaming the output in the required format
 - Updating final product metadata.

## Import modules

In [ ]:
import os
import shutil
import gdal
import numpy as np
from osgeo import gdal, gdalconst

In [ ]:
site = "dhm02"
dir_ = "mosaic_g"
directory = r"D:\uav\2022"

In [ ]:
os.chdir(r"{0}\{1}\{2}".format(directory, site, dir_))

In [ ]:
ls

## Composite greyscale images

In [ ]:
#cd Z:\Scratch\Rob\code\draft\uav\uav_composite_image

In [ ]:
cd D:\code\uav_composite_image

### Composite Multi band UAV images

In [ ]:
%run run_composite_multi_bands_image.py \
--directory Z:\Scratch\uav\2022\wmc03\mosaic_n  \
    --outfile Z:\Scratch\uav\2022\wmc03\mosaic_n\p4cj_wmc03_20220402_recnm3.tif

## Clip raster with extent shapefile

In [ ]:
os.listdir(r"{0}\{1}\mosaic_g".format(directory, site))

In [ ]:
if not os.path.isdir(r"{0}\{1}\shp".format(directory, site)):   
    os.mkdir(r"{0}\{1}\shp".format(directory, site))
    
os.listdir(r"{0}\{1}\shp".format(directory, site))

## Multispectral Composite Clip

In [ ]:
#cd C:\Users\rmcgr\Rangeland_monitoring\image_processing

In [ ]:
cd D:\code\image_processing

In [ ]:
%run clip_tool_rasterio.py \
--raster D:\uav\2022\wmc03\mosaic_n\p4cj_wmc03_20220402_recnm3.tif \
    --shpfile D:\uav\2022\wmc03\shp\wmc03_extent.shp \
        --outfile D:\uav\2022\wmc03\p4cj_wmc03_20220402_refnm3.tif

## RGB Clip

In [ ]:
#cd C:\Users\rmcgr\Rangeland_monitoring\image_processing

In [ ]:
cd D:\code\image_processing

In [ ]:
%run clip_tool_rasterio.py \
--raster D:\uav\2022\dhm02\mosaic_g\p4if_dhm02_g_mosaic_group1.tif \
    --shpfile D:\uav\2022\dhm02\shp\dhm02_extent.shp \
        --outfile D:\uav\2022\dhm02\p4if_dhm02_20220712_rgbnm2.tif

## Metadata update

### All UAV final products

In [ ]:
capture_date = '20220712'
pilot = 'Daniel Low Choy'
processing_date = "20220713"
utm = '52'
image_code = 'p4if'

## Multispec

In [ ]:
image = r"{0}\{1}\{3}_{1}_{2}_refnm{4}.tif".format(directory, site, capture_date, image_code, utm[1])
print(image)

## RGB

In [ ]:
image = r"{0}\{1}\{3}_{1}_{2}_rgbnm{4}.tif".format(directory, site, capture_date, image_code, utm[1])
print(image)

## Run metadata update

In [ ]:

#image = r"{0}\{1}\{3}_{1}_{2}_rgbnm2.tif".format(directory, site, capture_date, image_code)
print('Update metadata for: ',  image)
gdal.AllRegister()

inputdataset = gdal.Open(image)

inputdataset.SetMetadataItem("Flight","{0}".format(site))

inputdataset.SetMetadataItem("Flight date","{0}".format(capture_date))
inputdataset.SetMetadataItem("Pilot","{0}".format(pilot))

# platform and sensor
if image[-28:-27] == 'a':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","Micasense RedEdge-3")
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'b':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","Micasence RedEdge-M") 
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'c':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4 RTK01")
    inputdataset.SetMetadataItem("Sensor","DJI Multi WM331A")
    bands = "blue, green, red, red edge, nir"
elif image[-28:-27] == 'd':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")  
    bands = "blue, green, red"
elif image[-28:-27] == 'e':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A") 
    bands = "blue, green, red"
elif image[-28:-27] == 'g':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4 RTK01")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")
    bands = "blue, green, red"
elif image[-28:-27] == 'h':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4 RTK02")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A") 
    bands = "blue, green, red"
elif image[-28:-27] == 'i':
    inputdataset.SetMetadataItem("Platform","DJI Phantom 4 RTK02")
    inputdataset.SetMetadataItem("Sensor","DJI RGB WM331A")
    bands = "blue, green, red"
    
else:
    pass
    
inputdataset.SetMetadataItem("Processing date","{0}".format(processing_date))
                             
#datum                             
if utm == '52':                     
    inputdataset.SetMetadataItem("Datum","WGS 1984")
    inputdataset.SetMetadataItem("Projection","UTM zone 52S")
    inputdataset.SetMetadataItem("EPSG","32752")
elif utm == '53':
    inputdataset.SetMetadataItem("Datum","WGS 1984")
    inputdataset.SetMetadataItem("Projection","UTM zone 53S")
    inputdataset.SetMetadataItem("EPSG","32753") 
else:
    pass

gcp = image[-7:-6]
if gcp == 'g':
    inputdataset.SetMetadataItem("Geo-rectified","Yes")
elif gcp == 'n':
    inputdataset.SetMetadataItem("Geo-rectified","No")
else:
    pass

inputdataset.SetMetadataItem("Band composition","{0}".format(bands))

inputdataset = None

src = gdal.Open(image, gdalconst.GA_ReadOnly)

print(src.GetMetadata())


In [ ]:
shutil.rmtree(r"{0}\{1}\mosaic_n".format(directory, site))

In [ ]:
shutil.rmtree(r"{0}\{1}\mosaic_g".format(directory, site))